<a href="https://colab.research.google.com/github/Alby0n/Masked-Face-Recognition/blob/main/Colab/Masked_facerecogntion_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive

drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [6]:
%cd /content/gdrive/MyDrive/G-Drive

/content/gdrive/MyDrive/G-Drive


In [7]:
!pwd

/content/gdrive/MyDrive/G-Drive


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
from google.colab.patches import cv2_imshow
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import PIL
import cv2
%matplotlib inline
%load_ext autoreload
%autoreload 2

from tensorflow.keras.models import model_from_json

In [4]:


json_file = open('/content/gdrive/MyDrive/G-Drive/model_data/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('/content/gdrive/MyDrive/G-Drive/model_data/model.h5')
FRmodel = model
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

database = {}



def who_is_it(image_path, database, model):

    encoding = img_to_encoding(image_path,model)

    min_dist = 100

    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding - db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    # YOUR CODE ENDS HERE
    
    if min_dist > 1.2:
        #print("Not in the database.")
        pass
    else:
      pass
        #print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        #print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity





In [19]:
def most_common(List):
    return(mode(List))

def limitcnt(list5):
  x=most_common(list5)
  cnt = list5.count(x)
  if cnt==8 and gflag==False:
    flag=True
    gflag=True
def finalm(vid,zipfile): 
  
  directory="/content/gdrive/MyDrive/G-Drive/Temp_Folder"
  
  for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        imz=os.path.join(directory, filename)
        gi=img_to_encoding(imz,FRmodel)
        database[filename] = gi
    else:
        continue

  
  
  if os.path.exists('images'):
    shutil.rmtree('/content/gdrive/MyDrive/G-Drive/images')
  cam = cv2.VideoCapture(vid) # video i/p
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')  
  try:
      if not os.path.exists('images'):
          os.makedirs('images')
          print("Created")
  except OSError:
      print('Error: Creating directory of images')
  currentframe = 0
  while (True):
      ret, frame = cam.read()
      if ret:
          name = './images/frame' + str(currentframe) + '.jpg'
          print('Creating...' + name)
          cv2.imwrite(name, frame)
          currentframe += 1
      else:
          break
  cam.release()
  cv2.destroyAllWindows()

  base_dir = os.path.dirname('!pwd')
  prototxt_path = os.path.join(base_dir + 'model_data/deploy.prototxt')
  caffemodel_path = os.path.join(base_dir + 'model_data/weights.caffemodel')

  # Read the model
  model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)


  
  # Create directory 'faces' if it does not exist
  if not os.path.exists('faces'):
      print("New directory created")
      os.makedirs('faces')

  # Loop through all images and strip out faces
  count = 0
  Flg=False
  list5=[]
  flag=False
  gflag=False
  for file in os.listdir(base_dir + 'images'):
      file_name, file_extension = os.path.splitext(file)
      if (file_extension in ['.png','.jpg']):
          
          image = cv2.imread(base_dir + 'images/' + file)

          (h, w) = image.shape[:2]
          blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

          model.setInput(blob)
          detections = model.forward()
          try:
          # Identify each face
            for i in range(0, detections.shape[2]):
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                confidence = detections[0, 0, i, 2]

              # If confidence > 0.5, save it as a separate file
                if (confidence > 0.15):
                    count += 1
                    frame = image[startY:endY, startX:endX]
                    cv2.imwrite(base_dir + 'faces/'   + "gen.jpg", frame)
                  
                    nope,id=who_is_it(base_dir + 'faces/'   + "gen.jpg", database, FRmodel)
                    

                    if(float(nope)<1.2):
                      realMen="/content/gdrive/MyDrive/G-Drive/Temp Folder/"+id
                      realMen=cv2.imread(realMen)
                      Flg=True
                      return image,frame,realMen,id
                      

                      
                 

                      
          except:
            pass  
  if Flg==False:
    ss="None found in the database"
    fakeim=cv2.imread("/content/gdrive/MyDrive/G-Drive/1200px-No_Cross.svg.png")
    return fakeim,fakeim,fakeim,ss
    

#This is the USER INTERFACE

In [20]:
import zipfile
!pip install --quiet gradio
import gradio as gr
import numpy as np
import shutil
import os

def driverfunc(x,zipf,vid,y):
  if os.path.exists('Temp_Folder'):
    shutil.rmtree('/content/gdrive/MyDrive/G-Drive/Temp_Folder')

  if not os.path.exists('Temp_Folder'):
          os.makedirs('Temp_Folder')

  directory="/content/gdrive/MyDrive/G-Drive/Temp_Folder"
  tmp=zipf.name
  with zipfile.ZipFile(tmp, 'r') as zip_ref:
    zip_ref.extractall(directory)
  resultimg,resultframe,resultmen,resultid=finalm(vid,zipf.name)
  return [resultimg,resultframe,resultmen,resultid,"Program Terminated"]
    


iface = gr.Interface(
    fn=driverfunc,
    inputs=[
            gr.inputs.Textbox(lines=2,default="Upload The Zip Folder Containing the Images of People To Recognise Here", label=" "),
            gr.inputs.File(label="Zip Folder"),
            gr.inputs.Video(label="Upload the CCTV footage in here"),
            gr.inputs.Textbox(lines=2,default="Click Submit After Uploading the Files", label=" ")
            
            ],
    outputs=[gr.outputs.Image(label="CCTV frame in which the person was detected"),
             gr.outputs.Image(label="Detected face of Person"),
             gr.outputs.Image(label="Photograph stored in Database"),
             
             
             gr.outputs.Textbox(type="number"),
             gr.outputs.Textbox(label="Message:")],
    title="FACE RECOGNITION SYSTEM",
    theme="darkhuggingface",
    #allow_screenshot=False,
    allow_flagging=False,
    #live=True
     )
iface.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/interface.py:205: UserWarning: 'darkhuggingface' theme name is deprecated, using dark-huggingface instead.
  warnings.warn(f"'{theme}' theme name is deprecated, using {DEPRECATED_THEME_MAP[theme]} instead.")
/usr/local/lib/python3.7/dist-packages/gradio/interface.py:248: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn("The `allow_flagging` parameter in `Interface` now"


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://28483.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Created
Creating..../images/frame0.jpg
Creating..../images/frame1.jpg
Creating..../images/frame2.jpg
Creating..../images/frame3.jpg
Creating..../images/frame4.jpg
Creating..../images/frame5.jpg
Creating..../images/frame6.jpg
Creating..../images/frame7.jpg
Creating..../images/frame8.jpg
Creating..../images/frame9.jpg
Creating..../images/frame10.jpg
Creating..../images/frame11.jpg
Creating..../images/frame12.jpg
Creating..../images/frame13.jpg
Creating..../images/frame14.jpg
Creating..../images/frame15.jpg
Creating..../images/frame16.jpg
Creating..../images/frame17.jpg
Creating..../images/frame18.jpg
Creating..../images/frame19.jpg
Creating..../images/frame20.jpg
Creating..../images/frame21.jpg
Creating..../images/frame22.jpg
Creating..../images/frame23.jpg
Creating..../images/frame24.jpg
Creating..../images/frame25.jpg
Creating..../images/frame26.jpg
Creating..../images/frame27.jpg
Creating..../images/frame28.jpg
Creating..../images/frame29.jpg
Creating..../images/frame30.jpg
Creating..

KeyboardInterrupt: ignored